## Purpose:

Stack the rasters for each grid and calculate the standard deviation for each pixel.

In [ ]:
!pip install pandarallel --user

In [1]:
import matplotlib.pyplot as plt
from pandarallel import pandarallel
import rasterio as rio
import pandas as pd
import numpy as np
import glob
import os

In [3]:
index = pd.read_pickle('../../data/rasters.pkl')
print('Loaded', len(index), 'rasters')

Loaded 378 rasters


In [4]:
grids = pd.read_pickle('../../data/grid_data.pkl')
print('Loaded', len(grids), 'grids')

Loaded 1263 grids


#### Function to stack all of a grid's rasters.

Creates a numpy array of shape: (num_rasters, 501, 501)

Sorted temporally so that **stack[-1,:,:]** is the most recent image.

In [24]:
def stack_rasters(grid):
    '''
    Stacks the grid's rasters into one numpy array
    DIMENSIONS:
    0: time
    1: x
    2: y
    '''
    rasters = glob.glob('../../data/grids/' + str(grid.name) + '/*v3.0_dem.tif')  # grab all rasters
    rasters.sort(key=lambda x: x.split('_')[2])  # sort the rasters by date acquired


    arrays = []
    for raster in rasters:
        data = rio.open(raster).read(1) # Open Data        
        mask = rio.open(raster).read_masks(1).astype(float)  # Open Mask
        if data.shape != (501,501):
            plt.imshow(data)
            plt.show()
            return

#         # Calculate Amount of No Data (100 means no data at all)
#         _, counts = np.unique(mask, return_counts=True)
#         mask[mask==0] = np.nan
#         nodata = counts[0] / (counts[0] + counts[1]) * 100
        
        # Apply Laser Altimetry Correction
        raster_name = raster[raster.rfind('/')+1:raster.rfind('_')]
        data = data + index.loc[raster_name]['dz']
        data = data * mask / 255  # Mask the Data (no data becomes nan)
    
        arrays.append(data)
    
    stack = np.stack(arrays, axis=0)
    return stack

#### Function to calculate the standard deviation raster of a grid

Pixel-level standard deviation used to identify bodies of water

In [7]:
def get_std_raster(grid):
    outfile = '../../data/standard_deviation/' + str(grid.name) + '.tif'
    if os.path.exists(outfile):
        return
    
    stack = stack_rasters(grid)  # Stack the rasters 
    std = np.apply_along_axis(np.nanstd, 0, stack)  # Calculate standard deviation
    
    metadata = rio.open('./data/grids/' + str(grid.name) + '/' + grid['rasters'][0] + '_dem.tif').meta  # Get Metadata
    metadata.update({'dtype':'float64'})

    # Write raster to file
    with rio.open('./data/standard_deviation/' + str(grid.name) + '.tif', 'w', **metadata) as dst:
        dst.write(std, 1)
    return